# Project 2: CIFAR-10 Classification and FGSM Attack

In [ ]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt


In [ ]:
def load_batch(batch_file):
    with open(batch_file, 'rb') as f:
        entry = pickle.load(f, encoding='latin1')
        data = entry['data']
        labels = entry['labels']
        data = data.reshape(-1, 3, 32, 32).astype(np.uint8)
        data = np.transpose(data, (0, 2, 3, 1))
        return data, labels

class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def prepare_dataloaders(data_path, batch_size=128):
    train_data, train_labels = [], []
    for i in range(1, 6):
        data, labels = load_batch(os.path.join(data_path, f"data_batch_{i}"))
        train_data.append(data)
        train_labels += labels
    train_data = np.concatenate(train_data)
    test_data, test_labels = load_batch(os.path.join(data_path, "test_batch"))
    transform = transforms.Compose([transforms.ToTensor()])
    train_set = CIFAR10Dataset(train_data, train_labels, transform)
    test_set = CIFAR10Dataset(test_data, test_labels, transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


In [ ]:
class SimpleVGG(nn.Module):
    def __init__(self):
        super(SimpleVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [ ]:
def train_model(model, train_loader, test_loader, device, epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        model.train()
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")
        test_model(model, test_loader, device)

def test_model(model, loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, pred = torch.max(outputs, 1)
            correct += (pred == targets).sum().item()
            total += targets.size(0)
    acc = 100. * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return acc


In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    return torch.clamp(perturbed_image, 0, 1)

def run_fgsm_experiments(model, device, test_loader, epsilons=[0, 0.05, 0.1, 0.15, 0.2]):
    model.eval()
    results = []
    for eps in epsilons:
        correct = total = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data.requires_grad = True
            output = model(data)
            loss = F.cross_entropy(output, target)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            perturbed_data = fgsm_attack(data, eps, data_grad)
            output = model(perturbed_data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
        acc = correct / total
        results.append((eps, acc))
        print(f"Epsilon: {eps:.2f}, Accuracy: {acc:.4f}")
    return results


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader = prepare_dataloaders("cifar-10-batches-py")
model = SimpleVGG()
train_model(model, train_loader, test_loader, device, epochs=5)
fgsm_results = run_fgsm_experiments(model, device, test_loader)


In [ ]:
epsilons, accuracies = zip(*fgsm_results)
plt.plot(epsilons, accuracies, marker='o')
plt.title("FGSM Attack: Accuracy vs. Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()


In [ ]:
def adversarial_train_model(model, train_loader, test_loader, device, epsilon=0.1, epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        model.train()
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            data.requires_grad = True

            outputs = model(data)
            loss = criterion(outputs, targets)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data

            perturbed_data = fgsm_attack(data, epsilon, data_grad)
            outputs_adv = model(perturbed_data)
            loss_adv = criterion(outputs_adv, targets)
            optimizer.zero_grad()
            loss_adv.backward()
            optimizer.step()
        
        print(f"[Adv Training] Epoch {epoch+1}, Loss: {loss_adv.item():.4f}")
        test_model(model, test_loader, device)

# Train adversarially
adv_model = SimpleVGG()
adversarial_train_model(adv_model, train_loader, test_loader, device, epsilon=0.1, epochs=5)

# Evaluate robustness
adv_fgsm_results = run_fgsm_experiments(adv_model, device, test_loader)
